##### <span style="color:#483D8B; font-weight:bold;">Import Libraries</span>

In [ ]:
!pip install pyspellchecker
!pip install symspellpy
!pip install evaluate
!pip install --upgrade transformers
!pip install lime

In [ ]:
!pip install scipy==1.14.0

In [ ]:
!pip install pyLDAvis==2.1.2

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from symspellpy.symspellpy import SymSpell, Verbosity
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import warnings
import torch
import warnings
import time

In [ ]:
import sys
!{sys.executable} -m pip install praw

In [ ]:
from datasets import Dataset
from transformers import TrainingArguments
from transformers import Trainer
import evaluate
from transformers import EarlyStoppingCallback #to stop training early when eval loss stops improving
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaTokenizerFast, RobertaConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support #metric functions
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import spacy
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pyLDAvis.sklearn

##### <span style="color:#483D8B; font-weight:bold;">Import Dataset</span>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path2 = "/content/drive/MyDrive/Dataset-Hate-Speech-Detection.csv" # main dataset

In [ ]:
df = pd.read_csv(path2, low_memory=False) # low memory for more more accurate type inference

In [ ]:
df

Classification:

- 0 - Hate Speech
- 1 - Offensive Language
- 2 - Neither/Neutral content

In [ ]:
df.shape

NOTE: 24783 rows in original dataset

In [ ]:
df.reset_index(drop=True, inplace=True)
df.index = df.index + 0

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

NOTE: no null values & consistent datatypes

##### <span style="color:#483D8B; font-weight:bold;">Cleaning tweet column</span>

In [ ]:
from IPython.display import display
pd.set_option('display.max_colwidth', None)

In [ ]:
display(df.iloc[:10, :2])

In [ ]:
df[df['class'] == 0]

In [ ]:
df.groupby('class').nunique()

In [ ]:
class_count = df['class'].value_counts()

ax = class_count.plot(kind='bar')

for i, value in enumerate(class_count):
    ax.text(i, value + 0.5, str(value), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
data = [1430, 19190, 4163]
labels = ['Hate speech', 'Offensive Language', 'Neutral Language']

colors = ['#A9A9A9', '#B0C4DE', '#D2B48C']

plt.figure(figsize=(6, 6))
plt.pie(data, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Distribution by Class')
plt.show()

In [ ]:
display(df.iloc[:5, :2]) # .iloc[row_indexer, column_indexer]

In [ ]:
nltk.download('stopwords')

In [ ]:
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

dictionary_path = "/content/drive/MyDrive/frequency_dictionary_en_82_765.txt"  # path to the downloaded dictionary
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

#p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI)

stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def correct_spelling_symspell(text):
    corrected_words = []
    for word in text.split():
        # Skip short words and stopwords to speed up
        if len(word) < 3 or word in stop_words:
            corrected_words.append(word)
            continue

        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)
    return " ".join(corrected_words)

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (optional: keep the text if you want)
    text = re.sub(r'#\w+', '', text)
    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text).strip()
    text = correct_spelling_symspell(text)
    pattern = r'([{}])\1+'.format(re.escape(punctuation))
    text = re.sub(pattern, r'\1', text)
    tokens = tokenizer.tokenize(text)
    words = [w for w in tokens if w not in stop_words]
    return " ".join(words)

df['tweet_cleaned'] = df['tweet'].apply(clean_tweet)

In [ ]:
df['tweet_cleaned'] = df['tweet_cleaned'].str.replace(r'!\s*rt\s*:', '', regex=True).str.replace(r"[^\w\s.,!?']", "", regex=True)

In [ ]:
df['tweet_cleaned'] = df['tweet_cleaned'].str.replace(r'\brt\b', '', regex=True).str.strip() # added line after checking more common words in 'tweet_cleaned' column

In [ ]:
df

In [ ]:
df.groupby('class').nunique()

In [ ]:
df = df.drop_duplicates(subset='tweet_cleaned', keep='first').reset_index(drop=True)

In [ ]:
df.groupby('class').nunique()

In [ ]:
df.drop('tweet', inplace = True, axis=1)

In [ ]:
df = df.drop_duplicates(subset='tweet_cleaned', keep='first').reset_index(drop=True)

In [ ]:
class_count = df['class'].value_counts()

ax = class_count.plot(kind='bar')

for i, value in enumerate(class_count):
    ax.text(i, value + 0.5, str(value), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
df.head()

##### <span style="color:#483D8B; font-weight:bold;">Visualizations for insights with NLP techniques</span>


##### <span style="color:#483D8B; font-weight:bold;">Word cloud</span>

In [ ]:
for c in df['class'].unique():
  test = ' '.join(df[df['class'] == c]['tweet_cleaned'])
  wordcloud = WordCloud(width=800, height=800, background_color='white').generate(test)

  plt.figure(figsize=(8, 8), facecolor=None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad=0)

##### <span style="color:#483D8B; font-weight:bold;">Top Words per Class

In [ ]:
stop_words = set(stopwords.words('english'))

def get_top_words(texts, n=10):
    words = " ".join(texts).split()
    words = [re.sub(r'[^\w\s]', '', w.lower()) for w in words]
    words = [w for w in words if w and w not in stop_words]
    return Counter(words).most_common(n)

for c in df['class'].unique():
    top_words = get_top_words(df[df['class'] == c]['tweet_cleaned'])
    words, freqs = zip(*top_words)

    plt.figure(figsize=(8, 5))
    plt.barh(words, freqs, color='#A8D5BA')
    plt.gca().invert_yaxis()
    plt.title(f"Top Words in Class {c}")
    plt.xlabel("Frequency")
    plt.tight_layout()
    plt.show()

##### <span style="color:#483D8B; font-weight:bold;">Sentiment Analysis -1 (negative) to +1 (positive)

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
sia = SentimentIntensityAnalyzer() # VADER sentiment analyzer from NLTK

df['sentiment'] = df['tweet_cleaned'].apply(lambda x: sia.polarity_scores(x)['compound'])
print(df[['class', 'sentiment']].groupby('class').mean())

In [ ]:
# Sentiment values (replace with your actual values if needed)
sentiment_avg = df.groupby('class')['sentiment'].mean()

colors = ['crimson' if val < 0 else 'seagreen' for val in sentiment_avg]

plt.figure(figsize=(6, 6))
ax = sentiment_avg.plot(kind='bar', color=colors)

# Add sentiment values as text
for i, value in enumerate(sentiment_avg):
    va = 'bottom' if value >= 0 else 'top'
    y_offset = 0.02 if value >= 0 else -0.02
    ax.text(i, value + y_offset, f'{value:.2f}', ha='center', va=va, fontweight='bold')

plt.axhline(0, color='gray', linestyle='--')
plt.title('Average Sentiment per Class')
plt.ylabel('Sentiment Score')
plt.xticks(rotation=0)
plt.ylim(sentiment_avg.min() - 0.1, sentiment_avg.max() + 0.1)
plt.tight_layout()
plt.show()

##### TF-IDF / Term Frequency - Inverse Document Frequency

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', max_features=10)
tfidf_matrix = tfidf.fit_transform(df['tweet_cleaned'])

feature_names = tfidf.get_feature_names_out()
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
df_tfidf = df_tfidf[(df_tfidf != 0).any(axis=1)]  # filter out rows with all zeros

# Show most important words overall
print("Top TF-IDF words across all tweets:")
print(df_tfidf.sum().sort_values(ascending=False))

In [ ]:
# Your list of top words
top_words = ['bitch', 'bitches', 'like', 'hoes', 'pussy', 'hoe', 'nigga', 'ass', 'fuck', 'shit']

tfidf = TfidfVectorizer(vocabulary=top_words)
X = tfidf.fit_transform(df['tweet_cleaned'])
feature_names = tfidf.get_feature_names_out()

tfidf_df = pd.DataFrame(X.toarray(), columns=feature_names)

tfidf_df['class'] = df['class'].values

avg_tfidf_per_class = tfidf_df.groupby('class').mean()

plt.figure(figsize=(12, 6))
sns.heatmap(avg_tfidf_per_class, annot=True, cmap='Reds', fmt='.2f')
plt.title('Average TF-IDF of Top Words per Class')
plt.ylabel('Class')
plt.xlabel('Word')
plt.show()

In [ ]:
from matplotlib.patches import Circle

plt.figure(figsize=(10, 4))

ax = sns.heatmap(avg_tfidf_per_class, annot=True, cmap='Reds', fmt='.2f', cbar=False, yticklabels=False)

class_colors = ['red', 'orange', 'green']

for y, color in enumerate(class_colors):
    circle = Circle((-0.5, y + 0.5), 0.12, color=color, transform=ax.transData, clip_on=False)
    ax.add_patch(circle)

ax.set_ylabel('')
ax.set_yticks([])

plt.title('Avg. TF-IDF of Top Words per Class', fontsize=12)
plt.xlabel('Word', fontsize=10)
plt.xticks(rotation=45, ha='right', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
top_words = ['bitch', 'bitches', 'like', 'hoes', 'pussy', 'hoe', 'nigga', 'ass', 'fuck', 'shit']

rows = []

for word in top_words:
    for c in sorted(df['class'].unique()):
        examples = df[(df['class'] == c) & (df['tweet_cleaned'].str.contains(fr'\b{word}\b', case=False, na=False))]
        for tweet in examples['tweet_cleaned'].head(3):  # up to 3 examples
            rows.append({'word': word, 'class': c, 'tweet': tweet})

examples_df = pd.DataFrame(rows)

examples_df

##### Bigrams - Pairs of consecutive words

In [ ]:
# All classes
classes = sorted(df['class'].unique())

# Fit CountVectorizer on entire dataset to get vocabulary
vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words='english')
X = vectorizer.fit_transform(df['tweet_cleaned'])
bigrams = vectorizer.get_feature_names_out()
counts = X.toarray().sum(axis=0)

# Get top 10 bigrams overall
bigram_freq = sorted(zip(bigrams, counts), key=lambda x: x[1], reverse=True)[:10]
top_bigrams, _ = zip(*bigram_freq)

# Now get counts for these bigrams per class
class_counts = []
for c in classes:
    # Vectorize only tweets from class c, using the SAME vocabulary to align features
    Xc = CountVectorizer(ngram_range=(2, 2), stop_words='english', vocabulary=top_bigrams).fit_transform(df[df['class'] == c]['tweet_cleaned'])
    class_counts.append(Xc.toarray().sum(axis=0))

# Convert to numpy array for easier plotting: shape = (num_classes, num_bigrams)
class_counts = np.array(class_counts)

# Plot stacked bar chart
fig, ax = plt.subplots(figsize=(12, 6))

bottom = np.zeros(len(top_bigrams))
colors = ['tab:blue', 'tab:orange', 'tab:green']  # Adjust colors if you have more classes

for i, c in enumerate(classes):
    ax.barh(top_bigrams, class_counts[i], left=bottom, color=colors[i], label=f'Class {c}')
    bottom += class_counts[i]

ax.set_xlabel('Frequency')
ax.set_title('Top 10 Bigrams Frequency per Class (Stacked)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
phrase = "bitch ass"

# Filter rows where 'tweet_cleaned' contains the exact phrase (case-insensitive)
examples = df[df['tweet_cleaned'].str.contains(r'\b' + phrase + r'\b', case=False, na=False)]

# Select relevant columns: class and tweet
examples_df = examples[['class', 'tweet_cleaned']].rename(columns={'tweet_cleaned': 'tweet'})

examples_df

##### Topic Modeling using LDA - interactive pyLDAvis

In [ ]:
count_vec = CountVectorizer(stop_words='english')
X = count_vec.fit_transform(df['tweet_cleaned'])

lda = LatentDirichletAllocation(n_components=3, random_state=0)
lda.fit(X)

for i, topic in enumerate(lda.components_):
    words = [count_vec.get_feature_names_out()[index] for index in topic.argsort()[-10:]]
    print(f"Topic #{i}: {', '.join(words)}")

In [ ]:
_original_drop = pd.DataFrame.drop

def patched_drop(self, labels=None, axis=0, *args, **kwargs):
    return _original_drop(self, labels=labels, axis=axis, *args, **kwargs)

pd.DataFrame.drop = patched_drop

In [ ]:
doc_topic_dists = lda.transform(X)
topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, None]
doc_lengths = X.sum(axis=1).A1
vocab = count_vec.get_feature_names_out()
term_frequency = np.array(X.sum(axis=0)).flatten()

vis = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency
#

pyLDAvis.display(vis)

Note: the topics looks similar

##### <span style="color:#483D8B; font-weight:bold;">Model building</span>

##### <span style="color:#483D8B; font-weight:bold;">1 - Handling Imbalanced Data</span>

In [ ]:
multi_class_tweets = df.groupby('tweet_cleaned')['class'].nunique()

tweets_with_multiple_classes = multi_class_tweets[multi_class_tweets > 1]

print(tweets_with_multiple_classes) # the same tweet with multiple classes

In [ ]:
df_0 = df[df['class'] == 0]   # All hate speech tweets
df_1 = df[df['class'] == 1]   # All offensive tweets
df_2 = df[df['class'] == 2]   # All neutral tweets

In [ ]:
max_size = df_1.shape[0]
max_size

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['sentiment'])

In [ ]:
df.groupby('class').nunique()

Note: The model needs to be trained on additional data to predict subtle hate speech, various examples were checked at the evaluation stage, and the model couldn't spot them. New dataset to be added with subtle hatespeech examples (source huggingface)

In [ ]:
path3 = "/content/drive/MyDrive/subtle_hp_dataset.csv"

In [ ]:
df2 = pd.read_csv(path3, low_memory=False) # low memory for more more accurate type inference

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
# Apply the same cleanning process like in df

max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

dictionary_path = "/content/drive/MyDrive/frequency_dictionary_en_82_765.txt"  # path to the downloaded dictionary
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def correct_spelling_symspell(text):
    corrected_words = []
    for word in text.split():
        if len(word) < 3 or word in stop_words:
            corrected_words.append(word)
            continue

        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)
    return " ".join(corrected_words)

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (optional: keep the text if you want)
    text = re.sub(r'#\w+', '', text)
    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text).strip()
    text = correct_spelling_symspell(text)
    pattern = r'([{}])\1+'.format(re.escape(punctuation))
    text = re.sub(pattern, r'\1', text)
    tokens = tokenizer.tokenize(text)
    words = [w for w in tokens if w not in stop_words]
    return " ".join(words)

df2['tweet_cleaned'] = df2['cleaned_text'].apply(clean_tweet)

In [ ]:
df2 = df2.drop(columns=['cleaned_text'])
df2.head(2)

In [ ]:
df = pd.concat([df, df2], ignore_index=True)

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates(subset='tweet_cleaned', keep='first').reset_index(drop=True)

Note: Step completed to add new dataset to df, continuing with the balancing your data

In [ ]:
df.groupby('class').nunique() # total number matches

In [ ]:
# splitting again per class

df_0 = df[df['class'] == 0]   # All hate speech tweets
df_1 = df[df['class'] == 1]   # All offensive tweets
df_2 = df[df['class'] == 2]   # All neutral tweets

In [ ]:
df_0_upsampled = resample(df_0, replace=True, n_samples=max_size, random_state=42)
df_2_upsampled = resample(df_2, replace=True, n_samples=max_size, random_state=42)

In [ ]:
df_balanced = pd.concat([df_0_upsampled, df_2_upsampled, df_1])

In [ ]:
df_balanced.groupby('class').count()

In [ ]:
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_balanced.groupby('class').count()

In [ ]:
df_balanced.head(2)

In [ ]:
df_balanced["char_length"] = df_balanced["tweet_cleaned"].apply(len)
print(df_balanced["char_length"].describe()) # to find the max length for the model

##### <span style="color:#483D8B; font-weight:bold;">2 - RoBERTa model bulding</span>

Using a lighter version or Roberta, distilroberta-base

In [ ]:
df_reduced = df_balanced.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.8, random_state=42)).reset_index(drop=True) #changed from 0.7 to 0.8

In [ ]:
df_reduced = df_reduced.drop(columns=['char_length'])

In [ ]:
df_reduced.groupby('class').count()

In [ ]:
X = df_reduced["tweet_cleaned"]
y = df_reduced["class"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

In [ ]:
#tokenizer = RobertaTokenizerFast.from_pretrained("distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [ ]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=190) # changed the length (actuals length 147)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=190)

In [ ]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, y_train)
test_dataset = TweetDataset(test_encodings, y_test)

In [ ]:
config = RobertaConfig.from_pretrained(
        "distilroberta-base",
        num_labels=3,
        hidden_dropout_prob=0.3,                  # Increase dropout to help prevent overfitting
        attention_probs_dropout_prob=0.3          # Dropout in attention layers too
)

In [ ]:
#model = RobertaForSequenceClassification.from_pretrained("distilroberta-base", config=config)
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", config=config)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',  # Where to save model outputs and checkpoints
    num_train_epochs=8,     # Train up to 8 epochs - this is the best checkpoint (initially it was 10)
    per_device_train_batch_size=20, # Number of samples per batch for training
    per_device_eval_batch_size=20, # Number of samples per batch for evaluation
    logging_dir='./logs',          # Directory for training logs
    logging_steps=1000,             # Log training info every 1000 steps
    # no_cuda=True  # to force CPU usage if needed
    eval_strategy="epoch",              # Run evaluation after every epoch to check model performance
    save_strategy="epoch",                    # Save model checkpoint after each epoch
    load_best_model_at_end=True,              # After training, load the checkpoint with best evaluation loss!! (Check this again)
    metric_for_best_model="eval_loss",        # Use evaluation loss to determine which checkpoint is best
    weight_decay=0.01                         # Add weight decay (L2 regularization) to prevent overfitting
)

In [ ]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2                 # Stop training if eval loss does not improve for 2 consecutive epochs
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,          # function for evaluating metrics
    callbacks=[early_stopping],               # Enable early stopping during training
)

In [ ]:
trainer.train()

#### <span style="color:#483D8B; font-weight:bold;">3 - Classification report per class</span>

In [ ]:
from sklearn.metrics import classification_report

predictions = trainer.predict(test_dataset)

y_pred = predictions.predictions.argmax(axis=1)

y_true = predictions.label_ids

print(classification_report(y_true, y_pred, digits=4))

First results: [452/452 00:17]
{'eval_loss': 0.22755973041057587, 'eval_model_preparation_time': 0.0015, 'eval_runtime': 17.9547, 'eval_samples_per_second': 503.155, 'eval_steps_per_second': 25.174}

 Second results: [442/442 00:21]
{'eval_loss': 0.21381445229053497, 'eval_model_preparation_time': 0.0017, 'eval_runtime': 21.1788, 'eval_samples_per_second': 417.115, 'eval_steps_per_second': 20.87}

#### <span style="color:#483D8B; font-weight:bold;">4 - Save model</span>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save_pretrained('/content/drive/MyDrive/best_model')
tokenizer.save_pretrained('/content/drive/MyDrive/best_model')

In [ ]:
save_path = '/content/drive/MyDrive/best_model'
print(f"Model and tokenizer saved to: {save_path}")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/best_model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/best_model')

#### <span style="color:#483D8B; font-weight:bold;">5 - Predictions Test Data

In [ ]:
#reload model
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_path = "/content/drive/MyDrive/best_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
len(test_dataset)

In [ ]:
predictions = trainer.predict(test_dataset)

In [ ]:
logits = predictions.predictions
labels = predictions.label_ids

In [ ]:
predicted_labels = np.argmax(logits, axis=1)

predicted_labels

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Run prediction
# predictions = trainer.predict(test_dataset)

# Get predicted class labels
preds = np.argmax(predictions.predictions, axis=1)

# Get true labels
true_labels = predictions.label_ids

# Print classification report
print(classification_report(true_labels, preds))

# Confusion matrix
cm = confusion_matrix(true_labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='BuGn')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

CONCLUSIONS:

Class 0 (F1 = 0.95): high precision and very high recall.

Class 1 (F1 = 0.92): some true class 1 examples are being missed (recall = 0.89).

Class 2 (F1 = 0.97): both precision and recall are high.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('best_model')
tokenizer = AutoTokenizer.from_pretrained('best_model')

In [ ]:
print("Number of misclassified samples:", np.sum(preds != true_labels))

In [ ]:
X_test_reset = X_test.reset_index(drop=True)
misclassified_indices = np.where(preds != true_labels)[0]

print("Example misclassified tweet:", X_test_reset.iloc[misclassified_indices[0]])
print("True label:", true_labels[misclassified_indices[0]])
print("Predicted label:", preds[misclassified_indices[0]])

In [ ]:
from collections import defaultdict

misclassified_counts = defaultdict(int)
max_per_class = 10

for idx in np.where(preds != true_labels)[0]:
    true_class = true_labels[idx]
    pred_class = preds[idx]

    if misclassified_counts[true_class] < max_per_class:
        tweet_text = X_test_reset.iloc[idx]
        print(f"\nIndex {idx} | True: {true_class} | Pred: {pred_class}")
        print("Tweet:", tweet_text)

        misclassified_counts[true_class] += 1

    if all(count >= max_per_class for count in misclassified_counts.values()):
        break

In [ ]:
df[df['tweet_cleaned'].str.contains('julia roberts', case=False, na=False)] # predicted classification more accurate

#### <span style="color:#483D8B; font-weight:bold;">6 - validate example subtle hate speech

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("text-classification", model="/content/drive/MyDrive/best_model", tokenizer="/content/drive/MyDrive/best_model") # Load the pipeline with your saved model

texts = [
    "Some groups just don’t deserve the same rights as others.",
    "Oh sure, let's put another brilliant woman in charge, what could possibly go wrong.",
    "They’re always playing the victim card. You know the group I mean.",
    "I love how Asians are always good at math. Wish I had that gene!",
    "I identify as a toaster now. Respect my pronouns, lol.",
    "Some cultures just don’t value education like we do."
]

for text in texts:
    result = classifier(text)[0]
    print(f"Text: {text}")
    print(f"→ Class: {result['label']} | Confidence: {result['score']:.4f}")
    print("-" * 80)

CONCLUSION:

Model improved on identifying subtle hate speech but it needs further improvement as it misclassified some key example like "Text: I identify as a toaster now. Respect my pronouns, lol."

#### <span style="color:#483D8B; font-weight:bold;">Improt Data obtained via Reddit API

In [ ]:
file_path = '/content/drive/MyDrive/reddit_comments.csv'

df_rd = pd.read_csv(file_path)

df_rd.head()

In [ ]:
df_rd.shape

In [ ]:
df_rd.dtypes

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
# Apply the same cleanning process like in df

max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

dictionary_path = "/content/drive/MyDrive/frequency_dictionary_en_82_765.txt"  # path to the downloaded dictionary
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def correct_spelling_symspell(text):
    corrected_words = []
    for word in text.split():
        if len(word) < 3 or word in stop_words:
            corrected_words.append(word)
            continue

        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)
    return " ".join(corrected_words)

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (optional: keep the text if you want)
    text = re.sub(r'#\w+', '', text)
    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text).strip()
    text = correct_spelling_symspell(text)
    pattern = r'([{}])\1+'.format(re.escape(punctuation))
    text = re.sub(pattern, r'\1', text)
    tokens = tokenizer.tokenize(text)
    words = [w for w in tokens if w not in stop_words]
    return " ".join(words)

df_rd['cleaned_comment'] = df_rd['comment'].apply(clean_tweet)

#### <span style="color:#483D8B; font-weight:bold;">Predict class on Redit Data

In [ ]:
df_rd_2 = df_rd.drop(columns=['comment'])
df_rd_2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/best_model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/best_model')

# Set model to evaluation mode
model.eval()

In [ ]:
comments = df_rd['comment'].tolist()

inputs = tokenizer(comments, padding=True, truncation=True, return_tensors="pt") # Neural networks process fixed-length inputs efficiently.

In [ ]:
predictions = []
batch_size = 20  # smaller batches use less memory

for i in range(0, len(df_rd_2), batch_size):
    batch = df_rd_2['cleaned_comment'][i:i+batch_size].tolist()
    inputs = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)
    predictions.extend(preds.tolist())

df_rd_2['predicted_class'] = predictions
df_rd_2.head()

In [ ]:
df_rd_2['predicted_class'].value_counts().sort_index()

In [ ]:
df_combined = df_rd.drop(columns=['cleaned_comment']).join(df_rd_2)

In [ ]:
df_combined

In [ ]:
pd.set_option('display.max_colwidth', None)
df_combined[df_combined['predicted_class'] == 0].sample(10)

In [ ]:
sampled_df = df_combined.groupby('predicted_class').apply(lambda x: x.sample(30, random_state=42)).reset_index(drop=True)
sampled_df.to_csv('/content/drive/MyDrive/sampled_comments_per_class.csv')

#### <span style="color:#483D8B; font-weight:bold;">Create Streamlit App

In [ ]:
!pip install --upgrade transformers safetensors streamlit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

MODEL_PATH = "/content/drive/MyDrive/best_model"

@st.cache_resource
def load_model_and_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
    model.eval()
    return tokenizer, model

tokenizer, model = load_model_and_tokenizer()

device = torch.device("cpu")
model.to(device)

label_map = {0: "hate speech", 1: "offensive language", 2: "neutral language"}

st.title("Hate speech detector")

text = st.text_input("Enter text to classify:")

if st.button("Predict"):
    if not text.strip():
        st.warning("Please enter some text!")
    else:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        pred_idx = torch.argmax(outputs.logits, dim=1).item()
        st.write("Prediction:", label_map.get(pred_idx, "Unknown"))

In [ ]:
!pip install -q streamlit pyngrok transformers

from pyngrok import ngrok
import subprocess
import time

# Set your ngrok authtoken
NGROK_AUTH_TOKEN = "2zNQYRoaf4qEKJ2xl3D6iPlUxoy_sFhQusi2Qyio9acW1J7D"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start Streamlit in the background
proc = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Open a tunnel on port 8501
public_url = ngrok.connect(8501).public_url

print(f"🚀 Your Streamlit app is live at: {public_url}")

# Keep the cell alive to maintain the tunnel
while True:
    time.sleep(60)

#### <span style="color:#483D8B; font-weight:bold;">Create Streamlit App on local machine to be more stable for presenting - code below:

In [ ]:
!zip -r best_model_final.zip /content/drive/MyDrive/best_model

In [ ]:
from google.colab import files
files.download("/content/best_model_final.zip")

**Code in .py file:**

import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from pathlib import Path

MODEL_PATH = Path(r"C:\Users\User\Desktop\Ironhack_DA\Final-project-hate-speech-detection-with-NLP\best_model_final")

@st.cache_resource(show_spinner=False)
def load_model():
    model_path_posix = MODEL_PATH.as_posix()
    tokenizer = AutoTokenizer.from_pretrained(model_path_posix, local_files_only=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_path_posix, local_files_only=True)
    model.eval()
    return tokenizer, model

def predict(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    predicted_class_id = torch.argmax(probs).item()
    confidence = probs[0, predicted_class_id].item()
    return predicted_class_id, confidence

def main():
    st.title("Hate Speech Detector 🔍")

    tokenizer, model = load_model()

    user_input = st.text_area("Enter text to classify", height=150)

    if st.button("Predict"):
        if user_input.strip() == "":
            st.warning("Please enter some text to classify.")
        else:
            pred_class, conf = predict(user_input, tokenizer, model)
            st.write(f"**Predicted class:** {pred_class}")
            st.write(f"**Confidence:** {conf:.2f}")

if __name__ == "__main__":
    main()

Link to Streamlit App: http://localhost:8501/

## Via Hugging face cloud (Sreamlit_App.py):

import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

MODEL_REPO = "GeorgiosKoutroumanos/NLP-Roberta-HP-detection"

@st.cache_resource(show_spinner=False)
def load_model():
    st.write("Loading model from Hugging Face Hub...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_REPO)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_REPO)
    model.eval()
    st.write("Model loaded successfully!")
    return tokenizer, model

def predict(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    predicted_class_id = torch.argmax(probs).item()
    confidence = probs[0, predicted_class_id].item()
    return predicted_class_id, confidence

def main():
    st.title("Hate Speech Detector 🔍")

    tokenizer, model = load_model()

    user_input = st.text_area("Enter text to classify", height=150)

    if st.button("Predict"):
        if user_input.strip() == "":
            st.warning("Please enter some text to classify.")
        else:
            pred_class, conf = predict(user_input, tokenizer, model)
            st.write(f"**Predicted class:** {pred_class}")
            st.write(f"**Confidence:** {conf:.2f}")

if __name__ == "__main__":
    main()

https://final-project-hate-speech-detection-with-nlp.streamlit.app/

In [ ]:
import nbformat

# Full path to your notebook file
notebook_path = r"C:\Users\User\Desktop\Ironhack_DA\Final-project-hate-speech-detection-with-NLP\MAIN.ipynb"

# Load the notebook
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove widgets metadata if it exists
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Save the cleaned notebook
with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Notebook cleaned and saved successfully.")

In [ ]:
import nbformat

# Full path to your notebook file
notebook_path = r"C:\Users\User\Desktop\Ironhack_DA\Final-project-hate-speech-detection-with-NLP\MAIN.ipynb"

# Load notebook
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Check and delete corrupted widget metadata
widgets = nb.get("metadata", {}).get("widgets", {})
if "application/vnd.jupyter.widget-state+json" in widgets:
    print("Found widget metadata. Removing it...")
    del widgets["application/vnd.jupyter.widget-state+json"]
    # Clean up empty 'widgets' dict if necessary
    if not widgets:
        del nb["metadata"]["widgets"]

# Save the cleaned notebook
with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Notebook cleaned successfully.")
